In [1]:
if 'google.colab' in str(get_ipython()):
  print('Running on CoLab')
  %matplotlib inline
else:
    print('Not running on CoLab')
    %matplotlib notebook
    import os
    os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
    os.environ["CUDA_VISIBLE_DEVICES"] = "1"

Not running on CoLab


In [2]:
import time
from os import path

from matplotlib.animation import FFMpegWriter

from funtion_file import *
from ks_funtions import *
from tensorflow.keras.metrics import Precision, Recall

import keras as ks

In [3]:
row_len = 10
col_len = 12
kernel_size = (2, 2)
min_max_line_size = [(1,5),(1,6)]
rotate = True
num_of_mat = 500
fades_per_mat = 8
num_neuron = 256


matrix_lister = MatrixLister(row_len, col_len, kernel_size, min_max_line_size, rotate, num_of_mat, fades_per_mat)

In [4]:
checkpoint_filepath = 'weights.h5'
model_checkpoint_callback = ks.callbacks.ModelCheckpoint(filepath=checkpoint_filepath, save_weights_only=True,
                                                         monitor='val_precision', mode='max', save_best_only=True)

model = build_model(row_len, col_len, 32)
model.compile(optimizer='adam', loss=custom_loss, metrics=[Precision(name='precision'), Recall(name='recall')])

In [5]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 time_distributed (TimeDist  (None, 8, 10, 12, 32)     320       
 ributed)                                                        
                                                                 
 time_distributed_1 (TimeDi  (None, 8, 5, 6, 32)       0         
 stributed)                                                      
                                                                 
 time_distributed_2 (TimeDi  (None, 8, 5, 6, 64)       8256      
 stributed)                                                      
                                                                 
 time_distributed_3 (TimeDi  (None, 8, 2, 3, 64)       0         
 stributed)                                                      
                                                                 
 time_distributed_4 (TimeDi  (None, 8, 2, 3, 64)       1

In [6]:
batch_size = 2**9
epochs = 50

generator = DataGenerator(matrix_lister, batch_size, fades_per_mat, num_of_mat)

start = time.time()
hist = model.fit_generator(generator=generator,validation_data=generator, epochs=epochs, callbacks=[model_checkpoint_callback])

print(time.time() - start)

C:\Users\srben\AppData\Local\Temp\ipykernel_30560\1017878012.py:7: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  hist = model.fit_generator(generator=generator,validation_data=generator, epochs=epochs, callbacks=[model_checkpoint_callback])


Epoch 1/50
8/8 [==============================] - 5s 482ms/step - loss: 1.2085 - precision: 0.0403 - recall: 0.2129 - val_loss: 1.2059 - val_precision: 0.0411 - val_recall: 0.1445
Epoch 2/50
8/8 [==============================] - 3s 404ms/step - loss: 1.2040 - precision: 0.0407 - recall: 0.1238 - val_loss: 1.2014 - val_precision: 0.0400 - val_recall: 0.0513
Epoch 3/50
8/8 [==============================] - 3s 396ms/step - loss: 1.2009 - precision: 0.0374 - recall: 0.0200 - val_loss: 1.1993 - val_precision: 0.0384 - val_recall: 0.0042
Epoch 4/50
8/8 [==============================] - 3s 404ms/step - loss: 1.2276 - precision: 0.0374 - recall: 6.7026e-04 - val_loss: 1.2336 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00
Epoch 5/50
8/8 [==============================] - 3s 402ms/step - loss: 1.2307 - precision: 0.0000e+00 - recall: 0.0000e+00 - val_loss: 1.1780 - val_precision: 0.0466 - val_recall: 5.8203e-05
Epoch 6/50
8/8 [==============================] - 3s 409ms/step - loss: 1.10

In [7]:
if path.exists('weights.h5'):
    model.load_weights('weights.h5')

In [11]:
%matplotlib notebook
ani = matrix_lister.plot_matrices(model, 50)
plt.show()

1/1 [==============================] - 0s 17ms/step


<IPython.core.display.Javascript object>

In [9]:
if 'google.colab' in str(get_ipython()):
    f = r"animation.mp4"
    writervideo = FFMpegWriter(fps=24)
    ani.save(f, writer=writervideo)

In [10]:
plot_training_history(hist)

<IPython.core.display.Javascript object>